##### CODE:  using mRMR feature selection of the files: 

In [ ]:
# run the cell first for all setup

import re
from re import search
import os, sys
from os import walk
import traceback
import subprocess
import pandas as pd
import time
import math
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from autogluon.tabular import FeatureMetadata
from autogluon.tabular import TabularDataset, TabularPredictor

import mrmr

def column_del(name_columnD,df_temp_del, column_drop_list):
    for i, columndel in enumerate (column_drop_list):
        try:
            df_temp_del=df_temp_del.drop(columns=[columndel], axis=1)
            print ('Delete column : ', columndel ,'   -   ', name_columnD)
        except:     # KeyError
            print (' ')
    return (df_temp_del)



homepath=str(os.getcwd())





### CODE: Main


In [ ]:
timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()

ec50_label='ActivityValue_log_EC50'
AgonistAnagonist_label='AgonistP1AnagonistN1_label'
problem_type ='regression'  
eval_metric='mean_squared_error'

ECFP_index=[]
for i in range(0,1024):
    ECFPfeatures='ECFP_'+str(i)
    ECFP_index.append(ECFPfeatures)



workpath=homepath+'/Human_AgonistAntagonist/GPCR_to_test_FeatureSelection'
os.chdir(workpath)

print ('Now path = ', workpath)

os.chdir(workpath)
fileinforcuts=['GPCR_to_trainAgonistAntagonist_humanalltrainconcat.csv']
print('Doing: ',fileinforcuts,len(fileinforcuts))

for fileinforcut in fileinforcuts:
    fileforcut_data0=pd.read_csv(fileinforcut)
    fileforcut_data=fileforcut_data0.copy()
    fileforcut_data.replace([np.inf, -np.inf], np.nan, inplace=True) 
    fileforcut_data1=column_del(name_columnD='fileforcut_data',df_temp_del=fileforcut_data, column_drop_list=[ec50_label,'Protein_name','ChEMBL ID','Species','EfficacyValue_percentage'])

    for topK in [1,2,5,10,15,20,25,30,50,100,200,300,500,1000,1500,2000,2500,3000,3500,4500]:

        selected_features=[]
        print ('Doing topK : ',topK)
        mrmr_selected_features = mrmr.mrmr_regression(fileforcut_data1,fileforcut_data[ec50_label], K=topK)

        peotein_index= [p for p in mrmr_selected_features if 'Protein_code1' in p]
        padel_index= [p for p in mrmr_selected_features if 'padel1d2d' in p]

        selected_features.extend(peotein_index)
        selected_features.extend(padel_index)
        selected_features.extend(ECFP_index)

        selected_features.append('AgonistP1AnagonistN1_label')
        selected_features.append('ActivityValue_log_EC50')

        print ('selected_features: ',selected_features)

        fileforcut_data_features=fileforcut_data0[selected_features]

        testfeaturefileout=re.sub('.csv','',fileinforcut)+'_selefeatureTOP'+str(topK)+'.csv'
        os.chdir(workpath)
        fileforcut_data_features.to_csv(testfeaturefileout,index=False,encoding='utf-8')

    print ('Done MRMR selection: ', fileinMRMR)
    if (time.time()-starttime)/3600 > 1 :
        print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
    else:
        print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )

   






##### CODE:  FeatureSelection_GPCRtrain 


In [ ]:


import re
from re import search
import os, sys
from os import walk
import traceback
import subprocess
import pandas as pd
import time
import math
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from autogluon.tabular import FeatureMetadata
from autogluon.tabular import TabularDataset, TabularPredictor


os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"             # CPU only
os.environ["RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE"] = "1"

def column_del(name_columnD,df_temp_del, column_drop_list):
    for i, columndel in enumerate (column_drop_list):
        try:
            df_temp_del=df_temp_del.drop(columns=[columndel], axis=1)
        except:     # KeyError
            print (' ')
    return (df_temp_del)

def dataset_split_trainvalidtest (dataload, trainfilein):
    trainprotein=re.sub('\.csv','',trainfilein)
    traindata_split_filename=trainprotein+'_train.csv'
    validdata_split_filename=trainprotein+'_valid.csv'
    trainsize=0.8 # 0.85 or 0.7
    traindata, validdata = train_test_split(dataload, train_size=trainsize) #random_state=777,
    os.chdir(workpath)
    traindata.to_csv(traindata_split_filename,index=False,encoding='utf-8')
    validdata.to_csv(validdata_split_filename,index=False,encoding='utf-8')
    print ('Split dataset : ', trainfilein, ' -> ', traindata_split_filename,' & ', validdata_split_filename)
    return (traindata, validdata)

def AUTOGLUON_train(workpath,trainfilein,AIlabel_problem_eval,trainname):
    starttime1=time.time()
    os.chdir(workpath)
    AIlabel,problem_type,eval_metric,AgonistAnagonist_label=AIlabel_problem_eval
    trainname1=trainname+'_'
    trainname2=re.sub('GPCR_to_train','',trainname) # Agonist or Antagonist
    dataload=TabularDataset(trainfilein)
    dataload.replace([np.inf, -np.inf], np.nan, inplace=True) ###
    trainproteinname=re.sub(trainname1,'',trainfilein) 
    trainproteinname=re.sub(trainname,'',trainproteinname) 
    trainproteinname=re.sub('\.csv','',trainproteinname) 
    dataload=column_del(name_columnD=trainfilein,df_temp_del=dataload, column_drop_list=['Protein_name','ChEMBL ID','Species','EfficacyValue_percentage','level_0', 'index'])
    shape_x0,shape_y0=dataload.shape
    AGtrain=''
    if shape_x0 >8:
        AGtrain='True'        
        traindata,validdata=dataset_split_trainvalidtest(dataload,trainfilein)
        all_x,all_y=dataload.shape
        shape_x,shape_y =traindata.shape
        shape_x2,shape_y2 =validdata.shape
        timelimit = 'YES'  # 'No' 
        
        traintimelimit=int(shape_y/2)*10 

        if traintimelimit > 7*60*60:
            traintimelimit = 7*60*60  # - for all protein
        
        timestamp=time.strftime("%m%d",time.localtime(time.time()))
        print ('\n Now training: ',trainproteinname,' , training data size: ', shape_x, shape_y,' , test data size: ',shape_x2, shape_y2 , ' , \n Train time: ', (traintimelimit/60) ,' mins  at', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())) ,'\n\n\n')
        if timelimit == 'YES':
            if traintimelimit>60:
                float2time=int(traintimelimit/60)     
                ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_'+str(float2time)+'min'
            else:
                float2time=int(traintimelimit)
                ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_'+str(float2time)+'sec'
        else:
            timestamp=time.strftime("%Y%m%d_%H%M",time.localtime(time.time()))
            ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_nolimit'
        
        AGtrain=''
        AGtrain_time=0
        while AGtrain=='' : 
            try:
                predictor = TabularPredictor(label=AIlabel, problem_type=problem_type, eval_metric=eval_metric,path=ML_path, verbosity=3).fit(train_data=traindata, auto_stack=True, time_limit=traintimelimit, excluded_model_types=['KNN'])  
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Trained ', trainproteinname ,(time.time()-starttime1)/60, ' mins ', '\n\n' )
                AGtrain='True'
            except Exception as e:
                print('\n *** Error: ',e,' \n')
                print ('\n ***  REDO Autogluon train : ',AGtrain_time,' for the file: ', trainfilein)
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
                AGtrain_time=AGtrain_time+1
                traintimelimit=traintimelimit+1
                AGtrain=''
            if AGtrain_time > 7:
                AGtrain='False'
                leaderboard=pd.DataFrame()
                predictor=pd.DataFrame()
                print ('\n ***  Fail Autogluon train : ',AGtrain_time,' for the file: ', trainfilein,' -- NG -- ')
            
        if AGtrain=='True':
            try:
                leaderboard=predictor.leaderboard(validdata, extra_metrics=['mse','mae','pearsonr','rmse'], silent=True)
                leaderboard=leaderboard.sort_values(by=['score_val'],ascending=False)
                leaderboard=leaderboard.reset_index(drop=True)
                print ('\n\n',leaderboard,'\n Best model: ', predictor.get_model_best())
                leaderboardfile=ML_path+'/leaderboard_'+trainproteinname+'_size'+str(shape_x)+'.csv'
                os.chdir(workpath)
                leaderboard.to_csv(leaderboardfile,index=False,encoding='utf-8')
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
            except Exception as e:
                print('\n *** Error: ',e,' \n')
                print (' * CANNOT do evaluations -> skip', trainfilein,'\n')
                AGtrain='False'
                leaderboard=pd.DataFrame()
                predictor=pd.DataFrame()
    else:
        AGtrain='False'
        ML_path=''
        leaderboard=pd.DataFrame()
        predictor=pd.DataFrame()
        traintimelimit=0
        shape_x=int(shape_x0)
        shape_x2=0
        traintimelimit=0
        trainprotein=re.sub(r'.csv','',trainfilein)
        rename_trainfilein=trainprotein+'_smalldataset.csv'
        cmd='cp '+trainfilein+' '+rename_trainfilein
        print ('\n\n small dataset -> copy file: ', trainfilein, rename_trainfilein, '\n  ', cmd)
        os.chdir(workpath)
        subprocess.run(cmd, shell=True) #, capture_output=True)
        print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
    return(AGtrain,leaderboard,predictor,ML_path,shape_x0,shape_x,shape_x2,shape_y0,traintimelimit,dataload)

def TRAINevaluateEXPdata(workpath,predictor,ML_path,fileevaluation,AIlabel_problem_eval,df_protein_train,trainnum):  
    os.chdir(workpath)
    AIlabel= AIlabel_problem_eval[0]
    df_protein_train.at[trainnum,'file_evaluate']=fileevaluation
    df_protein_train.at[trainnum,'ML_path']=ML_path
    try:
        test_data=TabularDataset(fileevaluation)  #validdata
        y_test=test_data[AIlabel]
        test_data_nolab=column_del(name_columnD='test_data_nolab',df_temp_del=test_data, column_drop_list=[AIlabel])
        y_pred = predictor.predict(test_data_nolab)
        if problem_type=='binary':
            print ('sklearn accuracy_score : ',sklearn.metrics.accuracy_score(y_test,y_pred))
            print ('sklearn roc_auc_score : ',sklearn.metrics.roc_auc_score(y_test,y_pred))
            df_protein_train.at[trainnum,'sklearn accuracy_score']=sklearn.metrics.accuracy_score(y_test,y_pred)
            df_protein_train.at[trainnum,'sklearn roc_auc_score']=sklearn.metrics.roc_auc_score(y_test,y_pred)
        if problem_type=='regression':
            print ('sklearn mean_squared_error : ',sklearn.metrics.mean_squared_error(y_test,y_pred))
            print(f"sklearn Mean absolute percentage error: {sklearn.metrics.mean_absolute_percentage_error(y_test,y_pred) * 100:.3f} %")
            df_protein_train.at[trainnum,'sklearn mean squared error']=sklearn.metrics.mean_squared_error(y_test,y_pred)
            df_protein_train.at[trainnum,'sklearn mean absolute percentage error %']=sklearn.metrics.mean_absolute_percentage_error(y_test,y_pred)*100
    except:
        print ('No test sklearn')
    #######
    try:
        test_evaluate=predictor.evaluate(test_data)
        print (test_evaluate,'\n')
        df_protein_train.at[trainnum,'evaluate mean_squared_error']="{:.3f}".format(abs(test_evaluate['mean_squared_error']))
        df_protein_train.at[trainnum,'evaluate root_mean_squared_error']="{:.3f}".format(abs(test_evaluate['root_mean_squared_error']))
        df_protein_train.at[trainnum,'evaluate mean_absolute_error']="{:.3f}".format(abs(test_evaluate['mean_absolute_error']))
        df_protein_train.at[trainnum,'evaluate r2']="{:.2f}".format(abs(test_evaluate['r2']))
        df_protein_train.at[trainnum,'evaluate pearsonr']="{:.2f}".format(test_evaluate['pearsonr'])
        df_protein_train.at[trainnum,'evaluate median_absolute_error']="{:.3f}".format(abs(test_evaluate['median_absolute_error']))
    except:
        print ('No test evaluate')
    print('\n ********************************* \n')
    return (df_protein_train,trainnum)

def toTRAINfiles(workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval):
    os.chdir(workpath)
    ec50_label,problem_type,eval_metric,AgonistAnagonist_label= AIlabel_problem_eval
    print ('To train files: ', filenames, len(filenames))
    for i, trainfilein in enumerate (filenames):
        os.chdir(workpath)
        print ('\n\n ------------------------------ ',trainnum, trainfilein ,' ------------------------------ ')
        if re.search(substr[0], trainfilein):
            if re.search(substr[3], trainfilein):
                trainname=substr[3]
            elif re.search(substr[4], trainfilein):
                trainname=substr[4]
            elif re.search(substr[5], trainfilein):
                trainname=substr[5]
            trainname1=trainname+'_'
            trainproteinname=re.sub(trainname1,'',trainfilein)
            trainproteinname=re.sub('\.csv','',trainproteinname)
        df_protein_train.at[trainnum,'trainnum']="{:.0f}".format(trainnum)
        df_protein_train.at[trainnum,'Name']=trainproteinname
        trainname2=re.sub('GPCR_to_train','',trainname)
        df_protein_train.at[trainnum,'trainname']=trainname2
        df_protein_train.at[trainnum,'Time']=str(time.strftime("%Y%m%d_%H%M",time.localtime(time.time())))

        df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
        print ('\n Run AutoGluon: (',trainnum,')', trainfilein,' - ' ,trainproteinname)
        print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')

        AGtrain,leaderboard,predictor,ML_path,shape_x0,shape_x,shape_x2,shape_y0,traintimelimit,dataload=AUTOGLUON_train(workpath,trainfilein,AIlabel_problem_eval,trainname)

        peotein_index= [p for p in dataload.columns if 'Protein_code1' in p]
        padel_index= [p for p in dataload.columns if 'padel1d2d' in p]
        ECFP_index= [p for p in dataload.columns if 'ECFP' in p]
        
        if AGtrain=='True':
            
            try:
                print (trainfilein, ' size ',shape_x,'\n BEST model: ',predictor.get_model_best(),', score: ', "{:.2f}".format(leaderboard.at[0,'score_val'] ,' \n'))
                df_protein_train.at[trainnum,'ALL_size']="{:.0f}".format(shape_x0)
                df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
                df_protein_train.at[trainnum,'TEST_size']="{:.0f}".format(shape_x2)
                df_protein_train.at[trainnum,'features']="{:.0f}".format((shape_y0-1))
                df_protein_train.at[trainnum,'peotein_index']="{:.0f}".format(len(peotein_index))
                df_protein_train.at[trainnum,'padel_index']="{:.0f}".format(len(padel_index))
                df_protein_train.at[trainnum,'ECFP_index']="{:.0f}".format(len(ECFP_index))
                df_protein_train.at[trainnum,'BEST_model']=predictor.get_model_best()
                df_protein_train.at[trainnum,'score_val']=abs(leaderboard.at[0,'score_val'])              
                df_protein_train.at[trainnum,'mean_squared_error']=abs(leaderboard.at[0,'mean_squared_error'])
                df_protein_train.at[trainnum,'pearsonr']=leaderboard.at[0,'pearsonr']
                df_protein_train.at[trainnum,'mean_absolute_error']="{:.5f}".format(abs(leaderboard.at[0,'mean_absolute_error']))
                df_protein_train.at[trainnum,'root_mean_squared_error']="{:.5f}".format(abs(leaderboard.at[0,'root_mean_squared_error']))
                df_protein_train.at[trainnum,'score_test']="{:.5f}".format(leaderboard.at[0,'score_test'])
                df_protein_train.at[trainnum,'score_val']="{:.5f}".format(leaderboard.at[0,'score_val'])
                df_protein_train.at[trainnum,'traintimelimit_mins']=(traintimelimit/60)
            except:
                df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
                df_protein_train.at[trainnum,'features']="{:.0f}".format((shape_y0-1))
                df_protein_train.at[trainnum,'peotein_index']="{:.0f}".format(len(peotein_index))
                df_protein_train.at[trainnum,'padel_index']="{:.0f}".format(len(padel_index))
                df_protein_train.at[trainnum,'ECFP_index']="{:.0f}".format(len(ECFP_index))
                df_protein_train.at[trainnum,'BEST_model']='skip'
                
        else:
            print ('\n skip \n\n', trainnum, trainfilein)
            df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
            df_protein_train.at[trainnum,'features']="{:.0f}".format((shape_y0-1))
            df_protein_train.at[trainnum,'BEST_model']='skip'
        df_protein_train.at[trainnum,'File_Name']=str(trainfilein)
        trainnum=trainnum+1
        df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
        print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), ' done training: ', trainnum,trainfilein)

    os.chdir(workpath)
    df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
    print ('\n ********************************* \n',df_protein_train,'\n ********************************* \n')
    print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),  ' done training proteins \n', filenames, trainnum, '\n\n\n\n\n\n\n')
    return (df_protein_train,trainnum)

######################################## Main ########################################
timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()

homepath=str(os.getcwd())
workpath=homepath+'/Human_AgonistAntagonist/GPCR_to_test_FeatureSelection'
os.chdir(workpath)


ec50_label='ActivityValue_log_EC50'
AgonistAnagonist_label='AgonistP1AnagonistN1_label'
problem_type ='regression'  
eval_metric='mean_squared_error'
AIlabel_problem_eval=[ec50_label,problem_type,eval_metric,AgonistAnagonist_label]
substr=['GPCR_to_train','_train.csv','_valid.csv','GPCR_to_trainAgonistAntagonist','GPCR_to_trainAgonist','GPCR_to_trainAntagonist']

trainname=substr[3]

train_result_summary_file='protein_train_leaderboard_evaluate_sklearn_'+trainname+'_TOPfeatures_1024ECFP_AgonistP1AnagonistN1_'+str(timestamp)+'.csv'



workpath=str(os.getcwd())
print ('Now path = ', workpath)
os.chdir(workpath)

df_protein_train=pd.DataFrame() 

trainnum=0 

################################### to train seperate ##########################################
filenames=[]
pp2skip=[]
trainname1=trainname+'_'
folderfiles=(sorted(next(walk(workpath))[2], reverse = True))
for i, file in enumerate (folderfiles):
    if re.search(substr[1], file):
        continue
    if re.search(substr[2], file):
        continue
    if re.search(trainname1, file):
        if re.search('humanalltrainconcat_selefeatureTOP', file):
            filenames.append(file)
        
filenames=sorted(filenames) 
print (filenames,len(filenames))
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)

print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')

##########################################

if (time.time()-starttime)/3600 > 1 :
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
else:
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )


 



##### result data:

In [7]:
df_protein_train=pd.read_csv('protein_train_leaderboard_evaluate_sklearn_GPCR_to_trainAgonistAntagonist_TOPfeatures_1024ECFP_AgonistP1AnagonistN1_202306172246.csv')
df_protein_train

,trainnum,Name,trainname,Time,ALL_size,TRAIN_size,TEST_size,features,peotein_index,padel_index,ECFP_index,BEST_model,score_val,mean_squared_error,pearsonr,mean_absolute_error,root_mean_squared_error,score_test,traintimelimit_mins,File_Name
0,0,humanalltrainconcat_selefeatureTOP1,AgonistAntagonist,20230617_2246,100938,80750,20188,1026,1,0,1024,WeightedEnsemble_L3,-0.33066,0.431066,0.891307,0.42635,0.65656,-0.43107,85.500000,GPCR_to_trainAgonistAntagonist_humanalltrainco...
1,1,humanalltrainconcat_selefeatureTOP10,AgonistAntagonist,20230617_2358,100938,80750,20188,1035,7,3,1024,WeightedEnsemble_L3,-0.26876,0.316383,0.921557,0.35775,0.56248,-0.31638,86.333333,GPCR_to_trainAgonistAntagonist_humanalltrainco...
2,2,humanalltrainconcat_selefeatureTOP100,AgonistAntagonist,20230618_0116,100938,80750,20188,1125,82,18,1024,WeightedEnsemble_L3,-0.22597,0.253850,0.937673,0.31303,0.50383,-0.25385,93.833333,GPCR_to_trainAgonistAntagonist_humanalltrainco...
3,3,humanalltrainconcat_selefeatureTOP1000,AgonistAntagonist,20230618_0240,100938,80750,20188,1998,628,345,1024,WeightedEnsemble_L3,-0.21676,0.244902,0.939981,0.30946,0.49488,-0.24490,166.500000,GPCR_to_trainAgonistAntagonist_humanalltrainco...
4,4,humanalltrainconcat_selefeatureTOP15,AgonistAntagonist,20230618_0519,100938,80750,20188,1040,11,4,1024,WeightedEnsemble_L3,-0.25402,0.295617,0.926912,0.33582,0.54371,-0.29562,86.666667,GPCR_to_trainAgonistAntagonist_humanalltrainco...
5,5,humanalltrainconcat_selefeatureTOP1500,AgonistAntagonist,20230618_0638,100938,80750,20188,2382,812,545,1024,WeightedEnsemble_L3,-0.21521,0.244698,0.940062,0.30953,0.49467,-0.24470,198.500000,GPCR_to_trainAgonistAntagonist_humanalltrainco...
6,6,humanalltrainconcat_selefeatureTOP2,AgonistAntagonist,20230618_0942,100938,80750,20188,1027,1,1,1024,WeightedEnsemble_L3,-0.32401,0.430554,0.891380,0.42238,0.65617,-0.43055,85.666667,GPCR_to_trainAgonistAntagonist_humanalltrainco...
7,7,humanalltrainconcat_selefeatureTOP20,AgonistAntagonist,20230618_1058,100938,80750,20188,1045,15,5,1024,WeightedEnsemble_L3,-0.24929,0.289730,0.928433,0.33177,0.53827,-0.28973,87.166667,GPCR_to_trainAgonistAntagonist_humanalltrainco...
8,8,humanalltrainconcat_selefeatureTOP200,AgonistAntagonist,20230618_1217,100938,80750,20188,1225,162,38,1024,WeightedEnsemble_L3,-0.22252,0.249059,0.938877,0.30933,0.49906,-0.24906,102.166667,GPCR_to_trainAgonistAntagonist_humanalltrainco...
9,9,humanalltrainconcat_selefeatureTOP2000,AgonistAntagonist,20230618_1353,100938,80750,20188,2721,947,749,1024,WeightedEnsemble_L3,-0.21487,0.245169,0.939936,0.30958,0.49515,-0.24517,226.833333,GPCR_to_trainAgonistAntagonist_humanalltrainco...
